# Star-Wars Aura Demo Notebook 
## GDS Client version

## Overview
To demonstrate how to connnect to a Neo4j instance running in Aura.  This version uses the graphdatascience client.

### Created: September 11, 2022
### Author:  Mark Quinsland

### Additional resources:
Some of this material is based on the excellent, highly-recommended "Bite-Sized Neo4j for Data Scientiests" series of notebooks and youtube clips by CJ Sullivan. 

#### Note on Python Drivers:
For some of the notebooks in CJ's series, the official Neo4j Python driver named 'neo4j' is used.  In others, the community-supported 'py2neo' driver is used.  This notebook uses the 'neo4j' driver.  Since the publishing of the Bite-Sized series, Neo4j released a new Python client named 'graphdatascience' that supports additional Graph Data Science features.   Examples using the new graphdatascience library are available from the github repo containing this file.  

https://github.com/cj2001/bite_sized_data_science

https://youtu.be/Niys6g6NFfw?list=PL9Hl4pk2FsvVShoT5EysHcrs-hyCsXaWC

https://medium.com/@cj2001



In [1]:
# Client import
from graphdatascience import GraphDataScience


In [3]:
#aura connection details


 
# Replace with the actual URI, username, and password
AURA_CONNECTION_URI = "neo4j+s://ed23282c.databases.neo4j.io"
AURA_USERNAME = "neo4j"
AURA_PASSWORD = "nimda"

# Client instantiation
gds = GraphDataScience(
    AURA_CONNECTION_URI,
    auth=(AURA_USERNAME, AURA_PASSWORD),
    aura_ds=False
)

UnableToConnectError: Unknown function 'gds.version' (line 1, column 8 (offset: 7))
"RETURN gds.version()"
        ^

In [4]:
conn = Neo4jConnection(uri='neo4j+s://ed23282c.databases.neo4j.io', user='neo4j', pwd='nimda')

In [7]:
result = conn.query('MATCH (n) RETURN COUNT(n) AS ct')
# print(result)
# Consume the result object
print(result[0]['ct'])

325


### Utility class for Neo4j connection

In [3]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response